In [7]:
# import the required libraries
import pandas as pd
import numpy as np
import urllib.request
from bs4 import BeautifulSoup

# assigning the required link to variable url and requesting html data
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page=urllib.request.urlopen(url)
soup=BeautifulSoup(page,"lxml")

# using beautifulSoup to find the coorect table
right_table=soup.find('table', class_='wikitable sortable')

In [8]:
# creating empty lists to fill in the data from table
A=[]
B=[]
C=[]

# for loop to iterate through each row and fill each cell of the row with table data
for row in right_table.findAll("tr"):
    cells=row.findAll("td")
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

# converting the lists to dataFrame
df=pd.DataFrame(A,columns=['Postcode'])
df['Borough']=B
df['Neighbourhood']=C

# removing rows where Boroughs are 'Not Assigned'
df_e=df.drop(df[(df['Borough']=='Not assigned')].index)

# merging 'Neighbourhood' column for same 'Postcode' and seperating by commas
df_g=df_e.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).str.replace('(\\n)', '').reset_index()

# replacing 'Neighbourhood' cells having values 'Not Assigned' to 'Borough' values
df_g.loc[df_g.Neighbourhood == 'Not assigned', 'Neighbourhood'] = df_g['Borough']

In [4]:
# obtaining the csv file with geospatial coordinates
!wget -q -O can_geo.csv https://cocl.us/Geospatial_data;

In [11]:
# extracting csv data into dataFrame and changing 'Post Code' column name to 'Postcode' to perform merge with 'df_g' dataFrame
can_df = pd.read_csv("can_geo.csv")
can_df=can_df.rename(columns = {'Postal Code':'Postcode'})

# innerjoin between two dataFrames with unique column 'Postcode'
df3 = pd.merge(df_g, can_df)
df3.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
